# Initial Setup

## Libraries

In [1]:
import json
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import agent_tools

## Setups

In [2]:
# OpenAI Setup
load_dotenv(override=True)
openai = OpenAI()

# Pushover Setup
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [3]:
# The function will push message in the form of a payload to the phone based on predetermined parameters. 
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

# Tools and Handler

## Tools

In [4]:
# function if whether the user would like to be in touch with us
def record_user_details(email, name="name not provided.", notes='notes not provided.'):
    push(f"Recording interest from name: {name}, email: {email}, notes: {notes}")
    return {"recorded:": "ok"}

# function if the user doesn't know a question i doesn't know to answer. 
def record_unknown_question(question):
    push(f"Recording question: {question}")
    return {"recorded": "ok"}

## Handler

In [5]:
def handle_tool_calls(tool_calls):
    '''
    A function which takes in a list of tool calls, and executes them (sends a notification to phone) based on their prompt.
    Furthermore, each tool is already pre-mapped to a function, all this function does it search and execute like a 'dumb' agent. 
    '''
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        # finds function in the program based on the tool name. 
        tool = globals()[tool_name]
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results

# Development

## PDF-Reader

In [40]:
pdf_path = r"C:\Users\kaith\Documents\Github\Career-Agent\3_linkedln\LinkedlnProfilePDF.pdf"
website_path = r"C:\Users\kaith\Documents\Github\Career-Agent\3_linkedln\LinkedlnProfileWebsite.txt"

In [ ]:
reader = PdfReader(pdf_path)
linkedin = ""
website = ""

# Reads the pdf and compiles into a singular string.
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text


# Reads the website link (i could paste the weblink here instead, however for sourcing purposes I will do it in a .txt file)
with open(website_path, 'r') as link:
    linkedln_link = link.readline()

# This can be your own name (replace with with Kailas Thonnithodi with your own name)
full_name = "Kailas Thonnithodi" 

'https://www.linkedin.com/in/kailas-thonnithodi/'

## System Prompt

In [ ]:
sys_prompt = f"You are acting as {full_name}. You are answering questions on {full_name}'s website, \
    particularly questions related to {full_name}'s career, background, skills and experience. \
    Your responsibility is to represent {full_name} for interactions on the website as faithfully as possible. \
    You are given a website link ({linkedln_link}) which has more information regarding my LinkedIn profile which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
    If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "